In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re
path = './data/'
p1= pd.read_csv(path+'users.csv')
p2= pd.read_csv(path+'books.csv')
p3= pd.read_csv(path+'train_ratings.csv')
p4= pd.read_csv(path+'sample_submission.csv')
p5= pd.read_csv(path+'test_ratings.csv') # test와 sample은 완벽하게 동일하다.

#조금 전처리됨
p6= pd.read_csv('/opt/ml/input/code/users_fillna.csv')
p7= pd.read_csv('/opt/ml/input/code/books_fillna_cate.csv') 
p8= pd.read_csv('/opt/ml/input/code/books_lang.csv') 

users  =p6.copy()
books  =p7.copy()
train  =p3.copy()
sample =p4.copy()
test   =p5.copy()

boks =p8.copy()

print('users shape: ', users.shape) 
print(users.columns)#유저id, 지역, 나이
print('books shape: ', books.shape)
print(books.columns)#isbn(책번호), 제목, 저자, 출판년도, 출판사, img주소, 언어, 범주, 요약, img경로
print('train shape: ', train.shape) 
print(train.columns)# 유저 id, 책isbn, 평점

users shape:  (68092, 3)
Index(['user_id', 'age', 'country'], dtype='object')
books shape:  (149570, 9)
Index(['isbn', 'book_title', 'book_author', 'year_of_publication', 'publisher',
       'language', 'category', 'summary', 'high_category'],
      dtype='object')
train shape:  (306795, 3)
Index(['user_id', 'isbn', 'rating'], dtype='object')


# 출판사 건드리기

In [2]:
books['publisher'] = books['publisher'].str.lower()
books['ublisher'] = books['publisher'].copy()
books['publisher'].value_counts()
#출판사가 10000개나 있나? 미쿡이라 그럴 수도 있나 싶지만, 그래도 정제를 해주면 좋을 것 같다. 
# books[books['isbn'].str.startswith('0143')]#['publisher']

harlequin                   3006
ballantine books            2322
pocket                      2274
penguin books               1943
bantam books                1938
                            ... 
pamoon press                   1
wilhelm goldmann               1
circustuff                     1
mermaid books                  1
american map corporation       1
Name: publisher, Length: 11428, dtype: int64

In [3]:
# books[books['isbn'].str.startswith('0143')]#['publisher']
books[books['publisher'].str.contains('quin') \
	& (books['publisher'] != 'Harlequin')
	]#['publisher']


,isbn,book_title,book_author,year_of_publication,publisher,language,category,summary,high_category,ublisher
331,0373030959,"Unfriendly Proposition (Harlequin Romance, No ...",Jessica Steele,1990.0,harlequin,NaN,1,NaN,psychology,harlequin
332,0373031106,"Fair Trial (Harlequin Romance, No 3110)",Elizabeth Duke,1991.0,harlequin,en,fiction,Fair Trial by Elizabeth Duke released on Jan 2...,fiction,harlequin
333,0373033605,Brief Encounter (Sealed With A Kiss) (Harlequi...,Catherine George,1995.0,harlequin,NaN,1,NaN,cook,harlequin
334,0373033923,Husband Material (Holding Out For A Hero) (Har...,Emma Goldrick,1995.0,harlequin,en,fiction,Husband Material by Emma Goldrick released on ...,fiction,harlequin
335,0373162669,Always A Bridesmaid (Harlequin American Romanc...,Julie Kistler,1988.0,harlequin,en,fiction,Always A Bridesmaid by Julie Kistler released ...,fiction,harlequin
...,...,...,...,...,...,...,...,...,...,...
148562,0373220960,Into The Night (Harlequin Intrigue No. 96),Jan Michaels,1988.0,harlequin,en,1,Into The Night by Jan Michaels released on Jun...,juvenile,harlequin
148676,0373805438,Solution: Seduction,Elizabeth Oldfield,2003.0,harlequin,NaN,1,NaN,fiction,harlequin
148838,0373117868,"Dark Apollo (Top Author) (Harlequin Presents, ...",Sara Craven,1995.0,harlequin,en,fiction,"Dark Apollo by Sara Craven released on Nov 24,...",fiction,harlequin
149448,0373152604,"Once Bitten, Twice Shy : Love's Sting; The Wed...",Catherine Spencer,1996.0,harlequin books,NaN,1,NaN,fiction,harlequin books


In [23]:
books[books['isbn'].str.startswith('0373')]['publisher'].unique()
#이 중에서 harlequin이 3000개 가까이 되고 나머지는 5000개였으니 2000개 중 일부가 이상한 이름으로 떠돌고 있다. 
#그래도 4자리로 한번 묶어볼까?

array(['Silhouette', 'Harlequin', 'Harlequin Books', 'harlequin',
       'Worldwide Library', 'Red Dress Ink', 'Steeple Hill',
       'Silhouette Books', 'Harlequin Enterprises Limited',
       'World Wide Publications', 'Gold Eagle', 'Pocket Books',
       'Harlequin Mills & Boon Ltd', 'Thorndike Pr', 'Luna',
       'Harlequin Sales Corp', 'Worldwide Mystery', 'Mills and Boon',
       'Harlequin Sales Corp (Mm)', 'Worldwide', 'HQN Books',
       'Gale Group', 'Thorndike Press', 'Harlequin Romance'], dtype=object)

In [2]:
from tqdm import tqdm
publisher_dict=(books['publisher'].value_counts()).to_dict()
publisher_count_df= pd.DataFrame(list(publisher_dict.items()),columns = ['publisher','count'])

publisher_count_df = publisher_count_df.sort_values(by=['count'], ascending = False)
modify_list = publisher_count_df[publisher_count_df['count']>1].publisher.values

modify_list

array(['Harlequin', 'Ballantine Books', 'Pocket', ..., 'Guild Publishing',
       'Andre Deutsch', 'Peppermind'], dtype=object)

In [3]:
for publisher in tqdm(modify_list):
	try:
		number = books[books['publisher']==publisher]['isbn'].apply(lambda x: x[:4]).value_counts().index[0]
		right_publisher = books[books['isbn'].apply(lambda x: x[:4])==number]['publisher'].value_counts().index[0]
		books.loc[books[books['isbn'].apply(lambda x: x[:4])==number].index,'publisher'] = right_publisher
	except:
		pass

100%|██████████| 5276/5276 [03:20<00:00, 26.34it/s]


In [5]:
books['ublisher'].value_counts()


harlequin                   3006
ballantine books            2322
pocket                      2274
penguin books               1943
bantam books                1938
                            ... 
pamoon press                   1
wilhelm goldmann               1
circustuff                     1
mermaid books                  1
american map corporation       1
Name: ublisher, Length: 11428, dtype: int64

In [20]:
books['publisher'].value_counts()[700:]

hodder headline australia         7
brunnen-verlag, gieã?â?en         7
bodley head                       7
nymphenburger                     7
university of notre dame press    7
                                 ..
pearson education                 1
luso brazilian books              1
kluwer academic pub               1
nihon geijutsu shuppansha         1
information                       1
Name: publisher, Length: 818, dtype: int64

In [39]:
books[books['isbn'].str.startswith('014')]['publisher'].unique()

array(['Penguin Books', 'Puffin Books', 'Penguin Books Ltd'], dtype=object)

In [153]:
books['publisher'].value_counts()

pocket                  7306
bantam                  7177
harlequin               5317
harpercollins           5043
st. martin's press      4043
                        ... 
kiepenheuer                1
pearson education          1
luso brazilian books       1
kluwer academic pub        1
information                1
Name: publisher, Length: 1497, dtype: int64

In [152]:
# 내용물을 들여다 보는 코드 마지막으로 나라
(books[books['publisher'].str.contains('reilly') \
	# & (books['publisher'] != 'vintage books usa') \
	& (books['publisher'] != 'wizards of the coast') \
	& (books['publisher'] != 'dutton books') \
	# & (books['publisher'] != 'performing arts') \
	# & (books['category'].str.contains('dis') == True) \
	# & (books['publisher'].str.contains('press') == False) \
	])\
	# .sample(10)

,isbn,book_title,book_author,year_of_publication,publisher,language,category,summary,high_category,ublisher
502,1565041992,The Day I Swapped My Dad for 2 Goldfish,Neil Gaiman,1998.0,o'reilly,NaN,1,NaN,humor,white wolf games studio
2872,1565115228,Pretty Good Joke Book: A Prairie Home Companio...,Garrison Keillor,2001.0,o'reilly,NaN,1,NaN,children,highbridge audio
2873,1565122798,Educating Esme: Diary of a Teacher's First Year,Esme Raji Codell,2001.0,o'reilly,en,biography autobiography,"A must-read for parents, new teachers, and cla...",biography,algonquin books of chapel hill
2874,1565840968,"\I Won't Learn from You\"": And Other Thoughts ...",Herbert Kohl,1995.0,o'reilly,en,education,A collection of essays explore the educator&#3...,education,new press
3001,1565842103,A Queer Reader,Patrick Higgins,1994.0,o'reilly,en,social science,A Queer Reader is a rich collection of quotes ...,science,new press
...,...,...,...,...,...,...,...,...,...,...
148807,1565297768,MS-DOS 6.2 QuickStart,Steve Konicki,1994.0,o'reilly,NaN,1,NaN,juvenile,que
148808,1565298667,"Windows QuickStart, 3.11 edition",Ron Person,1994.0,o'reilly,NaN,1,NaN,death,que
148826,1565847172,Jews for Buchanan: Did You Hear the One About ...,John Nichols,2001.0,o'reilly,NaN,1,NaN,others,new press
149220,1565844424,Pig Tales: A Novel of Lust and Transformation ...,Marie Darrieussecq,1998.0,o'reilly,en,fiction,A young woman who lands a position at a beauty...,fiction,new press


In [144]:
# 치환 코드. 미국
lst = ['dorling kindersley']
lst = '|'.join(lst)
books['publisher'].where(books['publisher'].str.contains(lst)==False, 'dorling kindersley', inplace=True)

In [154]:
publishing = pd.DataFrame(books['publisher'].value_counts()).reset_index()
publishing.columns = ['publisher','count']
others_list = publishing[publishing['count']<5]['publisher'].values
books['publisher'].where(books['publisher'].isin(others_list) == False, 'others', inplace=True)

In [157]:
books['publisher'].value_counts()

pocket                          7306
bantam                          7177
harlequin                       5317
harpercollins                   5043
st. martin's press              4043
                                ... 
max hueber verlag                  5
osterreichischer agrarverlag       5
atlas                              5
william rudolf sabbott             5
ludwig, mã?â¼nchen                 5
Name: publisher, Length: 753, dtype: int64

In [160]:
books.drop('ublisher', axis=1, inplace=True)

In [4]:
#이제 오랜 시간은 그만. 그냥 파일로 저장하자.
# !rm /opt/ml/input/code/books_puble.csv
if not os.path.isfile('/opt/ml/input/code/books_publeno.csv'):
	print('saving')
	books.to_csv('/opt/ml/input/code/books_publeno.csv', index=False)

saving
